L5: Automate Event Planning
In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [3]:
import os
from dotenv import load_dotenv
# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key

# Load the environment variables
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

LLM value is None


In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

LLM value is None


In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

LLM value is None


In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [18]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [19]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    verbose=True
)

2025-01-24 12:38:33,016 - 16808 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [21]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference venue San Francisco\"}"
## Tool Output: 
{'searchParameters': {'q': 'Tech Innovation Conference venue San Francisco', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Tech. Innovation Conference 2035 - NICER Consortium', 'link': 'https://www.nicerconsortium.org/events/tech-innovation-conference-2035', 'snippet': 'Buy Tickets Time & Location Aug 01, 2035, 10:00 AM – 6:00 PM Conference Center, San Francisco About The Event', 'position': 1}, {'title': 'Tech. Innovation Conference 2035 - Maden-Tek', 'link': 'https://www.maden-tek.com/en/events/tech-innovation-conference-2035', 'snippet': 'Tech. Innovation Conference 2035. Time & Location. 01 Aug 2035, 10:00 – 18:00. Conference Center, 500 Terry Francois St. San 

In [22]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '500 Terry Francois St. San Francisco, CA 94158',
 'booking_status': 'Tickets already on sale',
 'capacity': 0,
 'name': 'Conference Center'}


In [23]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Tech Innovation Conference Promotion Report

## Marketing Activities:

1. Utilized online search tools to identify relevant tech conferences in Wichita.
2. Researched the Ad Astra Technology Summit as the Tech Innovation Conference on October 2, 2024.
3. Identified key event details such as the location at Hyatt Regency Wichita and the focus on industry panels and digital skills development.
4. Recognized the opportunity for teachers to earn graduate credit through WSU.

## Attendee Engagement:

1. Plan to leverage the information gathered for targeted promotion to attract at least 500 potential attendees.
2. Will emphasize the benefits of attending the Ad Astra Technology Summit, such as industry insights and professional development opportunities.
3. Intend to reach out to educators, industry professionals, entrepreneurs, and tech enthusiasts to promote the event effectively.
4. Aim to create engaging marketing materials highlighting the event's value proposition and unique offerings.

By strategically leveraging this information, the Tech Innovation Conference aims to drive attendance and create a memorable and insightful experience for all participants.